In [1]:
import pandas
import numpy
import datetime

In [2]:
def convert_to_datetime(row, datecolName="dateStr", timeColName="timeStr"):
    currDateStr = str( int( row[datecolName] ) )
#     return currDateStr
    if row[timeColName] < 10:
        currTimeStr = "000" + str( int( row[timeColName] ) )
    elif row[timeColName] < 100:
        currTimeStr = "00" + str( int( row[timeColName] ) )
    elif row[timeColName] < 1000:
        currTimeStr = "0" + str( int( row[timeColName] ) )
    else:
        currTimeStr = str( int( row[timeColName] ) )
    return datetime.datetime.strptime( currDateStr\
                    + ":" + currTimeStr, "%Y%m%d:%H%M" )

In [3]:
# Read SAPS data file!
datFileName = "../data/processedSaps-rbsp.txt"
sapsDataDF = pandas.read_csv(datFileName, sep=' ')
# add dst_bins
dstBins = [ -150, -75, -50, -25, -10, 10 ]
sapsDataDF = pandas.concat( [ sapsDataDF, \
                    pandas.cut( sapsDataDF["dst_index"], \
                               bins=dstBins ) ], axis=1 )
sapsDataDF.columns = [ ["dateStr", "hour","sapsLat", \
                     "sapsMLT", "sapsVel", "radId", "poesLat",\
                     "poesMLT", "dst_date", "dst_index", "timeStr", "dst_bin"] ]
sapsDataDF = sapsDataDF.drop(["radId", "poesLat", "poesMLT", "dst_date", "hour"], 1)
# Round off MLT to nearest half
sapsDataDF["sapsMLTRounded"] = [ round(x * 2) / 2 \
                                for x in sapsDataDF["sapsMLT"] ]
# Add a date column
sapsDataDF['date']= sapsDataDF.apply( convert_to_datetime, axis=1 )
sapsDataDF['dateStr']= sapsDataDF['date'].apply( lambda x: x.strftime('%Y%m%d') )
sapsDataDF.head()

,dateStr,sapsLat,sapsMLT,sapsVel,dst_index,timeStr,dst_bin,sapsMLTRounded,date
0,20110107,56.5,17.7543,308.2077,-18.0,0,"(-25, -10]",18.0,2011-01-07
1,20110107,55.5,18.0147,224.1588,-18.0,0,"(-25, -10]",18.0,2011-01-07
2,20110107,56.5,17.8749,307.4328,-18.0,0,"(-25, -10]",18.0,2011-01-07
3,20110107,55.5,18.1324,222.4787,-18.0,0,"(-25, -10]",18.0,2011-01-07
4,20110107,56.5,17.9955,305.4201,-18.0,0,"(-25, -10]",18.0,2011-01-07


In [4]:
def convert_excel_sheet_datetime(row):
    if row["month"] < 10:
        currMonStr = "0" + str(row["month"])
    else:
        currMonStr = str(row["month"])
    if row["day"] < 10:
        currDtStr = "0" + str(row["day"])
    else:
        currDtStr = str(row["day"])
    if row["hour UT"] < 10:
        currHrStr = "0" + str(row["hour UT"])
    else:
        currHrStr = str(row["hour UT"])
    if row["minute UT"] < 10:
        currMntStr = "0" + str(row["minute UT"])
    else:
        currMntStr = str(row["minute UT"])
        
    currDateStr = "20" + str(row["year (20**)"]) + currMonStr + currDtStr + "-" + currHrStr + currMntStr
    return datetime.datetime.strptime( currDateStr, "%Y%m%d-%H%M" )

intDatesVAPDF = pandas.read_excel( "../data/LIST_SAPS.xlsx" )
intDatesVAPDF['date']= intDatesVAPDF.apply( convert_excel_sheet_datetime, axis=1 )
intDatesVAPDF['dateStr']= intDatesVAPDF['date'].apply( lambda x: x.strftime('%Y%m%d') )
intDatesVAPDF.head()

,Van Allen Probe,year (20**),month,day,hour UT,minute UT,date,dateStr
0,A,12,10,1,5,20,2012-10-01 05:20:00,20121001
1,B,12,10,1,9,50,2012-10-01 09:50:00,20121001
2,A,12,10,8,8,2,2012-10-08 08:02:00,20121008
3,B,12,10,8,13,1,2012-10-08 13:01:00,20121008
4,A,12,10,8,16,48,2012-10-08 16:48:00,20121008


In [5]:
conjSapsDF = pandas.merge( intDatesVAPDF,sapsDataDF, on=["dateStr"] )
conjDatesArray = conjSapsDF["dateStr"].unique()
numpy.savetxt( "../data/saps-commondates.csv", conjDatesArray, fmt='%s')